# TensorFlow 实现神经网络

## 神经网络参数与 TensorFlow 变量

- 变量`tf.Variable` 的作用就是保存和更新神经网络中的参数 。
- TensorFlow 中的变量也需要指定初始值
    - 初始值可以设置成随机数、 
    - 常数
    - 通过其他变量的初始值计算得到 。 
    


```python
# 会产生一个2x3的矩 阵，矩阵中的元素是均值为 0 ，标准差为 2 的随机数。
#  tf.random normal 函数可以通过参数 mean 来指定平均值，在没有指定时默认为 0
weights = tf.Variable(tf.random_normal([2,3],stddev = 2))
# 常数初始化
biases = tf.Variable(tf.zeros[3])
# 通过其他变量来初始化
w2 = tf.Variable(weights.initialized_value())
w3 = tf.Variable(weights.initialized_value() * 2.0)
```

![tfrandom.png](imgs/tfrandom.png)
![tfconstanfunc.png](imgs/tfconstanfunc.png)

TensorFlow 变量初始化的具体函数
- `w1.initializer` 对具体的变量来初始化，该方法需要一个一个的初始参数
- `tf.global_variables_initializer()`: 自动对所有变量进行初始化（推荐）

**变量和张量的关系**
- 变量的声明函数 tf.Variable 是一个运算，运算的结果是一个张量
- 变量只是一种特殊的张量

** TensorFlow 中集合（ collection ）的概念**
- 所有的变量都会被自动地加入到 `GraphKeys.VARIABLES` 这个集合中
- 通过 `tf.global_variables()`函数可以拿到 当前计算图上所有的变量
- ，可以通过变量声明函数中的 `trainable` 参数来区分需要优化的参数

**变量的属性**
- 维度（ shape ）
    - 维度在程序运行中是有可能改变的 ，但是需要通过设置参数 v alidate_shape= False 。
    - 但是很少去改变其维度

- 类型 （type） 
    -  一个变量在构建之后，它的类型就不能再改变了 

**为什么需要placehold**

每轮迭代中选取的数据都要通过常量来表示,那么 TensorFlow 的计算图将会太
大。因为每生成一个常量 ，TensorFlow 都会在计算图中增加一个节点。  TensorFlow 提供了 placeholder 机制用 于提供输入数据.placeholder 相 当于定义了一个位置，这个位置中的数据在程序运行时再指定。这样在程序中就不需要生成大量常量来提供输入数据.



In [17]:
import tensorflow as tf

# 声明参数w1,w2, seed 设定随机种子，保证每次运行结果的一致性
w1 = tf.Variable(tf.random_normal([2,3],stddev = 1,seed =1))
w2 = tf.Variable(tf.random_normal([3,1],stddev = 1,seed =1))

# 输入特征量，x:1x2的矩阵
x = tf.constant([[0.7,0.9]])

# 前向传播计算
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

#init_op = tf.global_variables_initializer()

# 创建会话，可是计算
with tf.Session() as sess:
    # 初始化每个变量，是第一步
    print(sess.run(w1.initializer))
    print(sess.run(w2.initializer))
  
    #或者如下初始化
    #sess.run(tf.global_variables_initializer())
    print(sess.run(y))
    
                 

None
None
Tensor("MatMul_6:0", shape=(1, 3), dtype=float32)
[[3.957578]]


In [19]:
import tensorflow as tf

# 声明参数w1,w2, seed 设定随机种子，保证每次运行结果的一致性
w1 = tf.Variable(tf.random_normal([2,3],stddev = 1,seed =1))
w2 = tf.Variable(tf.random_normal([3,1],stddev = 1,seed =1))

# 输入特征量，x:1x2的矩阵
#x = tf.constant([[0.7,0.9]])
# n 个样例
x = tf.placeholder(tf.float32,shape=(3,2),name = "input")

# 前向传播计算
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)
y = tf.sigmoid(y)

cross_entropy = -tf.reduce_mean(
    y_ * tf.log(tf.clip_by_value(y,le-10,1.0))
    + (1-y) * tf.log(tf.clip_by_value(1-y,1e_10,1.0))
)

train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

#init_op = tf.global_variables_initializer()

# 创建会话，可是计算
with tf.Session() as sess:
    # 初始化每个变量，是第一步
    print(sess.run(w1.initializer))
    print(sess.run(w2.initializer))
  
    #或者如下初始化
    #sess.run(tf.global_variables_initializer())
    print(sess.run(y,feed_dict = {x:[[0.7,0.9],[0.1,0.4],[0.5,0.8]]}))
    
                 

None
None
[[3.957578 ]
 [1.1537654]
 [3.1674924]]


### 训练神经网络的过程

1. 定义神经网络的结构和前向传播的输出结果。
2. 定义损失函数以及选择反向传播优化的算法。
3. 生成会话（ tf.Session ）并且在训练、数据上反复运行反向传播优化算法。


In [6]:
import tensorflow as tf
from numpy.random import RandomState

# 训练数据的batch大小
batch_size = 8

# 声明参数w1,w2, seed 设定随机种子，保证每次运行结果的一致性
w1 = tf.Variable(tf.random_normal([2,3],stddev = 1,seed =1))
w2 = tf.Variable(tf.random_normal([3,1],stddev = 1,seed =1))

# 在 shape 的一个维度上使用 None 可以方便使用不同的 batch 大小。在训练时需要把数据分
# 成比较小的 batch ， 但是在测试时，可以一次性使用全部的数据。当数据集比较小时这样比较
# 方便测试，但数据集比较大时，将大量数据放入一个 batch 吁能会导致内存溢出。
x  = tf.placeholder(tf.float32,shape=(None,2),name = "x_input")
y_ = tf.placeholder(tf.float32,shape=(None,1),name = "y_input")
# 前向传播计算
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)
y = tf.sigmoid(y)

cross_entropy = -tf.reduce_mean(
    y_ * tf.log(tf.clip_by_value(y,1e-10,1.0))
    + (1-y) * tf.log(tf.clip_by_value(1-y,1e-10,1.0))
)

train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

#init_op = tf.global_variables_initializer()

rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size,2)

Y = [[int(x1 + x2 <1)] for (x1,x2) in X]

# 创建会话，可是计算
with tf.Session() as sess:
    # 初始化每个变量，是第一步
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    print (sess.run(w1))
    print (sess.run(w2))
    
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start+batch_size, dataset_size)
        print("start,end",start,end)
        
        sess.run(train_step,
                feed_dict = {x:X[start:end],y_:Y[start:end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(
            cross_entropy,feed_dict={x:X,y_:Y}
            )
            print("%d:%g"%(i,total_cross_entropy))
            
            
    print (sess.run(w1))
    print (sess.run(w2))
    
           

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
start,end 0 8
0:0.314006
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,e

start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,en

start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,en

start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 

start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,en

start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,e

start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,en

start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,end 24 32
start,end 32 40
start,end 40 48
start,end 48 56
start,end 56 64
start,end 64 72
start,end 72 80
start,end 80 88
start,end 88 96
start,end 96 104
start,end 104 112
start,end 112 120
start,end 120 128
start,end 0 8
start,end 8 16
start,end 16 24
start,en